# Part 1: Simulating a single Star Population

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import pandas as pd
import tabulate

To ensure reproducibility, a fixed seed was set at the beginning of the Monte Carlo simulation.  
This means that all results are exactly reproducible with identical parameters.

In [ ]:
SEED = 42
np.random.seed(SEED)

In [ ]:
# Create figures directory to save plots
from pathlib import Path
FIGURES_DIR = Path("figures")
FIGURES_DIR.mkdir(exist_ok=True)

### 0. Import all the necessary functions from the source module

In [ ]:
from src.sampling import sample_salpeter, sample_metallicity
from src.stellar_physics import ms_lifetime_years, stellar_endpoint
from src.supernovae import is_type_ia, core_collapse_SN_type

### 1. Sampling a star population of N = 100000 
Masses and Metallicities are given **in relation to the sun's mass ☉**

In [ ]:
N = 100000

masses = sample_salpeter(N)
# Ensure masses is at least 1D array
masses = np.atleast_1d(masses)

metallicities = sample_metallicity(N)
# Ensure metallicities is at least 1D array
metallicities = np.atleast_1d(metallicities)

print("Sampled masses:", masses)
print("Sampled metallicities:", metallicities)

### 2. Using stellar physics to determine Lifetimes and Endstates
with:  
WD: white dwarf  
NS: neutron star  
BH: black hole

In [ ]:
lifetimes = np.array([
    ms_lifetime_years(masses, metallicities)
    for masses, metallicities in zip(masses, metallicities)
])
print("Computed lifetimes:", lifetimes)

endstates = np.array([
    stellar_endpoint(m) 
    for m in np.array(masses)
    ])
print("Determined stellar endpoints:", endstates)

### 3. Determining supernovae (and their types)
A star needs to have high mass (at least 8 x M☉) for its core to collapse.
Type Ia specifically is low probability.

In [ ]:
cc_sn_types = np.array([
    core_collapse_SN_type(m)
    for m in masses
    ])  
print("Determined core-collapse SN types:", cc_sn_types)

ia_flags = np.array([
    is_type_ia(m)
    for m in masses
    ])  
print("Determined Type Ia flags:", ia_flags)

sn_types = np.where(ia_flags, "Type Ia", cc_sn_types)
print("Determined supernova types:", sn_types)

### 4. Summary and Plots for the single population

In [ ]:
total_cc = np.sum(cc_sn_types != None)
total_Ia = np.sum(ia_flags)
total_SN = total_cc + total_Ia

print("===== SUMMARY =====")
print(f"Total stars: {N}")
print(f"Core-collapse SN:  {total_cc} ({total_cc/N*100:.3f} %)")
print(f"Type Ia SN:        {total_Ia} ({total_Ia/N*100:.3f} %)")
print(f"All supernovae:   {total_SN} ({total_SN/N*100:.3f} %)")

unique, counts = np.unique(sn_types[sn_types != None], return_counts=True)
print("\nSupernovae types:")
for u, c in zip(unique, counts):
    print(f"{u}: {c}")


In [ ]:
plt.figure(figsize=(14,4))

plt.subplot(1,3,1)
plt.hist(masses, bins=50)
plt.xlabel("mass [M☉]")
plt.title("Stellar Initial Masses")

plt.subplot(1,3,2)
plt.hist(np.log10(lifetimes), bins=50)
plt.xlabel("log10 lifetime [years]")
plt.title("Main Sequence Lifetime")

plt.subplot(1,3,3)
plt.bar(unique, counts)
plt.title("Supernova Types")
plt.xticks(rotation=20)

plt.tight_layout()
plt.savefig(FIGURES_DIR/"single_star_population_results.png", dpi=150)
plt.show()
plt.close()

# PART 2: Multiple Monte Carlo Runs: 
### How do the supernova rate and type distribution change depending on the initial conditions of a star population?

How does varying our comparitive parameters change our output?  
we will vary: **maximum mass** when sampling masses  
we will see: rare core-collapse sn will happen more often

(other interesting parameters we did not choose:  
varying binary fraction, IMF-function, distribution of metallicity, ia-efficiancy)

### 0. Import the necessary functions to make multiple Monte Carlo Runs

In [ ]:
from src.mc_functions import run_single_mc, mc_statistics


### 1. Run the Monte Carlo Simulation
For every m_max value the simulation we established above (with N=100000) is run 100 times.

In [ ]:
### THIS STEP TAKES ABOUT 2-4 MINUTES TO RUN ###

N = 100000
n_realizations = 100

m_max_values = [30, 50, 80, 120, 150, 200]

comparison_results = {}

for m_max in m_max_values:
    stats = mc_statistics(N=N, m_max=m_max, n_realizations=n_realizations)
    comparison_results[m_max] = stats


### 2. Results / Statistics  

(Even though type ia supernovae occur in our sampled population, they do not appear in  
the M C statistics, because binary systems are not included in this model, see `reports/supernovae-results`)

In [ ]:
for m_max, stats in comparison_results.items():
    print(f"\nm_max = {m_max} m_sun")
    for sn_type, values in stats.items():
        print(
            f"  {sn_type:15s} "
            f"mean={values['mean']:.1f}, "
            f"std={values['std']:.1f}, "
            f"CI68={values['ci_68']}, "
            f"CI95={values['ci_95']}"
        )


Aggregated Results:

In [ ]:
rows = []

for m_max, stats in comparison_results.items():
    cc = stats["core_collapse"]
    
    rows.append({
        "M_max": m_max,
        "CCSN_mean": cc["mean"],
        "CCSN_std": cc["std"],
        "CCSN_CI95_low": cc["ci_95"][0],
        "CCSN_CI95_high": cc["ci_95"][1]
    })

df_results = pd.DataFrame(rows)
df_results = df_results.sort_values("M_max").reset_index(drop=True)

df_results


### 3. Visualisation  
Correlation between maximum mass and amount of core-collapses

In [ ]:
plt.figure()
plt.plot(
    df_results["M_max"],
    df_results["CCSN_mean"],
    marker="o"
)

plt.xlabel(r"maximum star mass $M_{\mathrm{max}}$ [$M_\odot$]")
plt.ylabel("Average number of core-collapse supernovae")
plt.title("Core-collapse supernovae vs. maximum star mass")
plt.grid(True)
plt.savefig(FIGURES_DIR/"ccsn_vs_m_max.png", dpi=150)
plt.show()
plt.close()


In [ ]:
plt.figure(figsize=(8, 5))

plt.fill_between(
    df_results["M_max"].values,
    df_results["CCSN_CI95_low"].values,
    df_results["CCSN_CI95_high"].values,
    alpha=0.3,
    label="95% confidence interval"
)

plt.plot(
    df_results["M_max"].values,
    df_results["CCSN_mean"].values,
    marker="o",
    label="Mean"
)

plt.xlabel("Maximum stellar mass $M_{\\mathrm{max}}$ [$M_\\odot$]")
plt.ylabel("Number of core-collapse supernovae")
plt.title("Core-collapse supernovae vs. maximum stellar mass")

plt.legend()
plt.tight_layout()
plt.savefig(FIGURES_DIR/"ccsn_confidence_interval.png", dpi=150)
plt.show()
plt.close()


### 4. Interpretation

The statitics show an approximately linear relation between maximum stellar mass and amount of core-collaps supernovae.  
The results are compiled and interpreted in `reports/supernovae_results`.